# Python - Getting more information from Tracebacks
> A tutorial to get more information from Python exception stack traceback.

- toc: true 
- badges: true
- comments: true
- categories: [python]
- keywords: [python, traceback, exception, frame]
- image: images/2022-02-11-python-traceback.png

## About
This notebook demonstrates what the Python Traceback object is, and how can we get more information out of it to better diagnose exception messages.

### Credit
This blog post is based on an article originally written in `Python Cookbook` published by `O'Reilly Media, Inc.` and released July 2002. In book's chapter 15, there is a section with the title `Getting More Information from Tracebacks` written by `Bryn Keller`. An online version of this article is available at https://www.oreilly.com/library/view/python-cookbook/0596001673/ch14s05.html.

The original article uses Python 2.2, but I have adapted it for Python 3.8. Also, I have added some commentary to give more insights on Python Traceback object.

### Environment Details 

In [1]:
#collapse-hide
from platform import python_version

print("python==" + python_version())

python==3.8.5


## Discussion

Consider the following toy example where we are getting some data from an external source (an API call, a DB call, etc.), and we need to find the length of individual items provided in the list. We know that items in the list will be of type `str` so we have used a `len()` function on it.

We got an exception when we ran our function on received data, and now we are trying to investigate what caused the error.

In [2]:
#collapse-hide
# this is intentionally hidden as we don't know about the data received from an external source. 
data = ["1", "22", 333, "4444"]

In [3]:
##
# our toy example function.
import sys, traceback

def get_items_len(items: list) -> list:
    """
    this function returns the length of items received in a list.
    """
    items_len = []
    for i in items:
        items_len.append(len(i))
    
    return items_len

In [4]:
##
# let's run our function on "data" received from an external source
try:
    get_items_len(data)
except Exception as e:
    print(traceback.print_exc())

None


Traceback (most recent call last):
  File "<ipython-input-4-42cd486e1858>", line 4, in <module>
    get_items_len(data)
  File "<ipython-input-3-8421f841ba77>", line 11, in get_items_len
    items_len.append(len(i))
TypeError: object of type 'int' has no len()


We got an exception while data processing and the `Traceback` message gives us some details. It tells us that we have received some data of type _integer_ instead of _string_, and we are trying to call _len()_ function on it. But we don't know the actual data value that caused the exception, and we don't know the _index_ of the item in the list that caused this error. Depending on the use case, information about the local variables, or input data that caused the error can be crucial in diagnosing the root cause of an error.

Fortunately, all this information is already available to us in the Traceback object, but there are no built-in methods that give this information directly. Let us try some of the built-in methods on the Traceback object to see the kind of information we could get from them.

In [5]:
#collapse-output
# calling traceback module built-in methods
try:
    get_items_len(data)
except Exception as e:
    print("***** Exception *****")
    print(e)

    exc_type, exc_value, exc_traceback = sys.exc_info()
    print("\n***** print_tb *****")
    traceback.print_tb(exc_traceback, limit=1, file=sys.stdout)

    print("\n***** print_exception *****")
    # exc_type below is ignored on 3.5 and later
    traceback.print_exception(exc_type, exc_value, exc_traceback,
                                limit=2, file=sys.stdout)
    
    print("\n***** print_exc *****")
    traceback.print_exc(limit=2, file=sys.stdout)

    print("\n***** format_exc, first and last line *****")
    formatted_lines = traceback.format_exc().splitlines()
    print(formatted_lines[0])
    print(formatted_lines[-1])

    print("\n***** format_exception *****")
    # exc_type below is ignored on 3.5 and later
    print(repr(traceback.format_exception(exc_type, exc_value,
                                            exc_traceback)))
                                            
    print("\n***** extract_tb *****")
    print(repr(traceback.extract_tb(exc_traceback)))

    print("\n***** format_tb *****")
    print(repr(traceback.format_tb(exc_traceback)))

    print("\n***** tb_lineno *****", exc_traceback.tb_lineno)

***** Exception *****
object of type 'int' has no len()

***** print_tb *****
  File "<ipython-input-5-73d5b316a567>", line 4, in <module>
    get_items_len(data)

***** print_exception *****
Traceback (most recent call last):
  File "<ipython-input-5-73d5b316a567>", line 4, in <module>
    get_items_len(data)
  File "<ipython-input-3-8421f841ba77>", line 11, in get_items_len
    items_len.append(len(i))
TypeError: object of type 'int' has no len()

***** print_exc *****
Traceback (most recent call last):
  File "<ipython-input-5-73d5b316a567>", line 4, in <module>
    get_items_len(data)
  File "<ipython-input-3-8421f841ba77>", line 11, in get_items_len
    items_len.append(len(i))
TypeError: object of type 'int' has no len()

***** format_exc, first and last line *****
Traceback (most recent call last):
TypeError: object of type 'int' has no len()

***** format_exception *****
['Traceback (most recent call last):\n', '  File "<ipython-input-5-73d5b316a567>", line 4, in <module>\n    

All these methods are useful but we are still short on information about the state of local variables when the system crashed. Before writing our custom function to get the variables state at the time of exception, let us spend some time to understand the working of Traceback object.

### Traceback Module
> https://docs.python.org/3/library/traceback.html <br>

This module provides an easy-to-use interface to work with `traceback objects`. It provides multiple functions that we can use to extract the required information from traceback. So far, we have used methods from this module in the above examples.


### Traceback Objects
> https://docs.python.org/3/reference/datamodel.html <br>
> On this page search for term "Traceback objects"

Traceback objects represent a stack trace of an exception. A traceback object is implicitly created when an exception occurs and may also be explicitly created by initializing an instance of class `types.TracebackType`. _traceback_ object is also an instance of _types.TracebackType_ class. When an exception occurs, a traceback object is initialized for us, and we can obtain it from any of the following two methods. 
1. It is available as a third item of the tuple returned by sys.exc_info() "`(type, value, traceback)`"
2. It is available as the `__traceback__` object of the caught exception. "`Exception.__traceback__`"

A traceback object is a linked list of nodes, where each node is a `Frame object`. Frame objects form their own linked list but in the opposite direction of traceback objects. Together they work like a doubly-linked list, and we can use them to move back and forth in the stack trace history. It is the frame objects that hold all the stack's important information. traceback object has some special attributes
* `tb_next` point to the next level in the stack trace (towards the frame where the exception occurred), or `None` if there is no next level
* `tb_frame` points to the execution frame of the current level
* `tb_lineno` gives the line number where the exception occurred

In [6]:
##
# method 1: get traceback object using sys.exc_info()
try:
    get_items_len(data)
except Exception as e:
    print(sys.exc_info()[2])

In [7]:
##
# method 2: get traceback object using Exception.__traceback__
try:
    get_items_len(data)
except Exception as e:
    print(e.__traceback__ )

If there is no exception in the system, then calling sys.exc_info() will only return `None` values.

In [8]:
##
# no exception is generated so sys.exc_info() will return None values.
try:
    get_items_len(['1','2','3','4'])
except Exception as e:
    print(sys.exc_info()[2])

### Frame Objects
> https://docs.python.org/3/reference/datamodel.html <br>
> On this page search for term "Frame objects"

Frame objects represent execution frames. It has some special attributes
* `f_back` is a reference to the previous stack frame (towards the caller), or None if this is the bottom stack frame
* `f_code` is the code object being executed in this frame. We will discuss `Code Objects` in next the section
* `f_lineno` is the current line number of the frame — writing to this from within a trace function jumps to the given line (only for the bottom-most frame). A debugger can implement a Jump command (aka Set Next Statement) by writing to f_lineno. This attribute will give you the line number in the code on which exception occurred
* `f_locals` is a dictionary used to lookup local variables. From this dictionary we can get all the local variables and their state at the time of exception
* `f_globals` is a dictionary for global varaibles

### Code Objects
> https://docs.python.org/3/reference/datamodel.html <br>
> On this page search for term "Code Objects"

Code objects represent byte-compiled executable Python code or bytecode. Some of its attributes include
* `co_name` gives the function name being executed
* `co_filename` gives the filename from which the code was compiled

There are many other helpful attributes in this object, and you may read about them from the docs.

### Visual representation of Traceback, Frame and Code Objects
![](images/2022-02-11-python-traceback.png) 

*<center>figure 1: Visual representation of Traceback, Frame and Code Objects</center>*

### Custom fuction for additional exception info
Now with this additional information on stack trace objects, let us create a function to get variables state at the time of exception.

In [9]:
#collapse-show
def exc_info_plus():
    """
    Provides the usual traceback information, followed by a listing of all the
    local variables in each frame.
    """
    tb = sys.exc_info()[2]

    # iterate forward to the last (most recent) traceback object.
    while 1:
        if not tb.tb_next:
            break
        tb = tb.tb_next
    stack = []

    # get the most recent traceback frame
    f = tb.tb_frame

    # iterate backwards from recent to oldest traceback frame 
    while f:
        stack.append(f)
        f = f.f_back
    
    # stack.reverse() # uncomment to get innermost (most recent) frame at the last

    # get exception information and stack trace entries from most recent traceback object
    exc_msg = traceback.format_exc()

    exc_msg += "\n*** Locals by frame, innermost first ***"
    for frame in stack:
        exc_msg += f"\nFrame {frame.f_code.co_name} in {frame.f_code.co_filename} at line {frame.f_lineno}"
        for key, value in frame.f_locals.items():
            exc_msg += f"\n\t {key:20} = "
            try:
                data = str(value)
                # limit variable's output to a certain number. You can adjust it as per your requirement.
                # But not to remove it as output from large objects (e.g. Pandas DataFrame) can be troublesome. 
                output_limit = 50
                exc_msg += (data[:output_limit] + "...") if len(data) > output_limit else data
            except:
                exc_msg += "<ERROR WHILE PRINTING VALUE>"

    return exc_msg

In [10]:
#collapse-output
#now let us try our custom exception function and see the ouput
try:
    get_items_len(data)
except Exception as e:
    print(exc_info_plus())

Traceback (most recent call last):
  File "<ipython-input-10-01264d9e470a>", line 4, in <module>
    get_items_len(data)
  File "<ipython-input-3-8421f841ba77>", line 11, in get_items_len
    items_len.append(len(i))
TypeError: object of type 'int' has no len()

*** Locals by frame, innermost first ***
Frame get_items_len in <ipython-input-3-8421f841ba77> at line 11
	 items                = ['1', '22', 333, '4444']
	 items_len            = [1, 2]
	 i                    = 333
Frame <module> in <ipython-input-10-01264d9e470a> at line 6
	 __name__             = __main__
	 __doc__              = Automatically created module for IPython interacti...
	 __package__          = None
	 __loader__           = None
	 __spec__             = None
	 __builtin__          = <module 'builtins' (built-in)>
	 __builtins__         = <module 'builtins' (built-in)>
	 _ih                  = ['', '#collapse-hide\nfrom platform import python_...
	 _oh                  = {}
	 _dh                  = ['/data/_note

Note the output from the first stack frame in the above stack trace. It is easy now to see (_items_) that we received in our function. The item at index _i_ is also available (333) on which our function crashed. Using our custom function unexpected errors are logged in a format that makes it a lot easier to find and fix the errors. Let's fix our function to handle unexpected integer values.

In [11]:
##
# let's fix our function to handle unexpected 'int' items by converting them to 'str'
def get_items_len(items: list) -> list:
    """
    this function returns the length of items received in a list.
    """
    items_len = []
    for i in map(str, items):
        items_len.append(len(i))
    
    return items_len

# test it again
get_items_len(data)

[1, 2, 3, 4]